In [2]:
import os
import sys
import re
import json
from datasets import (load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict,
    Value,
    Features
)

In [3]:
sys.path.append('../')

In [4]:
import torch
import random
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt

In [5]:
from transformers import AutoTokenizer
from tqdm.notebook import tqdm

## Tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained('hyunwoongko/kobart', use_fast=True)

/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [7]:
print('Size of tokenizer : %d' %len(tokenizer))

Size of tokenizer : 30000


## Dataset

In [8]:
paper_dataset = load_dataset('metamong1/summarization_paper', 
    use_auth_token='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')

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c)


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
law_dataset = load_dataset('metamong1/summarization_law', 
    use_auth_token='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')

Reusing dataset law_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___law_summarization/Paper Summarization/1.2.0/b422baca30e481895dd2b572a7ff9f6c6428725e575fdafb73c0aa1d62356973)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
magazine_dataset = load_dataset('metamong1/summarization_magazine', 
    use_auth_token='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')

Reusing dataset magazine_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___magazine_summarization/Magizine Summarization/1.0.0/506cb41eb0b96b084eafa5dd5fe3b51ff0d1061256700adf1aa92d3b19762c36)


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
news_dataset = load_dataset('metamong1/summarization_news', 
    use_auth_token='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')

Reusing dataset news_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___news_summarization/News Summarization/1.0.0/ae25c3215dc878e979d01f1157dbfb014c0a6985fc959ae45eaf10847db75600)


  0%|          | 0/2 [00:00<?, ?it/s]

## Documents

In [12]:
news_docs = [data['text'] for data in news_dataset['train']]
law_docs = [data['text'] for data in law_dataset['train']]
magazine_docs = [data['text'] for data in magazine_dataset['train']]
paper_docs = [data['text'] for data in paper_dataset['train']]

In [13]:
total_docs = news_docs + law_docs + magazine_docs + paper_docs
random.shuffle(total_docs)

## Preprocessing

In [14]:
outrange_comp = re.compile('[^\u3040-\u30ff\
    \uac00-\ud7af\
    \u4e00-\u9fff\
    \u0000-\u007f\
    \u2000-\u206f\
    \u25a0-\u25ff]') 

In [15]:
outrange_docs = [doc for doc in tqdm(total_docs) if outrange_comp.search(doc) != None]

  0%|          | 0/390689 [00:00<?, ?it/s]

In [16]:
print('Size : %d' %len(outrange_docs))

Size : 68669


In [18]:
def base_preprocess(doc) :
    doc = outrange_comp.sub(' ', doc)
    doc = re.sub('\s+', ' ', doc)
    return doc.strip()

In [19]:
preprocessed_docs = [base_preprocess(doc) for doc in tqdm(total_docs)]

  0%|          | 0/390689 [00:00<?, ?it/s]

In [55]:
paper_title = [data['title'] for data in paper_dataset['train']]